In [1]:
import numpy as np
import pandas as pd
from gensim.models.keyedvectors import KeyedVectors
from nltk.corpus import stopwords
from sklearn.ensemble import RandomForestClassifier
import sklearn.metrics
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.svm import SVC
from nltk.tokenize import word_tokenize
from gensim.models import Word2Vec
from sklearn.metrics import classification_report
import nltk
nltk.download('wordnet')
nltk.download('stopwords')
nltk.download('punkt')
import re

#Preparing Data
we will get rid of unused columns which are irrelevant for this project

In [53]:
dataset1 = np.array(pd.read_csv('/Users/joacosoulez/Downloads/transcriptions/dev_split_Depression_AVEC2017.csv',delimiter=',',encoding='utf-8'))[:, 0:2]
dataset2 = np.array(pd.read_csv('/Users/joacosoulez/Downloads/transcriptions/full_test_split.csv',delimiter=',',encoding='utf-8'))[:, 0:2]
dataset3 = np.array(pd.read_csv('/Users/joacosoulez/Downloads/transcriptions/train_split_Depression_AVEC2017.csv',delimiter=',',encoding='utf-8'))[:, 0:2]


In [54]:
dataset = np.concatenate((dataset1, np.concatenate((dataset2, dataset3))))


In [55]:
X_train = np.zeros((1,74))
Y_train = []

X_test = np.zeros((1,74))
Y_test = []

In [56]:
def make_data(X):
    for i in range(X.shape[0]):
        if(X[i,1] == 0):
            X[i,0] = 0
            for j in range(7):
                X[i,j+1] = 0
    X = np.array(X)
    X = np.average(X, axis = 0)
    X = np.array(X.reshape(1, -1))
    return X

In [108]:
for i in range(len(dataset2)):
        fileName = "/Users/joacosoulez/Desktop/audio/Covarep/"+str(i)+"_COVAREP.csv"
        data = pd.read_csv(fileName,header = None)
        X_temp = data.iloc[:,:].values
        X_temp = make_data(X_temp)
        X_test = np.concatenate((X_test,X_temp),0)
        for n in range(len(dataset2)):
            if dataset2[n][0] == i:
                Y_test.append(dataset2[n][1])
    

In [117]:
X_test = np.delete(X_test,0,0)
Y_test = np.array(Y_test)
count_0 = 0
count_1 = 0

In [134]:
for i in range(len(dataset3)):
        fileName = "/Users/joacosoulez/Desktop/audio/Covarep/"+str(i)+"_COVAREP.csv"
        data = pd.read_csv(fileName,header = None)

for n in range(len(dataset3)):
    if(dataset3[n][1] == 0):
      count_0 +=1
      if(count_0<27):
        X_temp = data.iloc[:,:].values
        X_temp = make_data(X_temp)
        X_train = np.concatenate((X_train,X_temp),0)
        if dataset3[n][0] == i:
            Y_train.append(dataset3[n][1])
    else:
      count_1 +=1
      if(count_1<27):
        X_temp = data.iloc[:,:].values
        X_temp = make_data(X_temp)
        X_train = np.concatenate((X_train,X_temp),0)
        if dataset3[n][0] == i:
            Y_train.append(dataset3[n][1])

In [135]:
Y_train

[]

In [150]:
count_0 = 0
count_1 = 0    

for i in range(len(dataset3)):
    fileName = "/Users/joacosoulez/Desktop/audio/Covarep/"+str(i)+"_COVAREP.csv"
    data = pd.read_csv(fileName,header = None)
    for n in range(len(dataset1)):
        if(dataset1[n][1] == 0):
            count_0 += 1
            if(count_0<27):
                X_temp = data.iloc[:,:].values
                X_temp = make_data(X_temp)
                X_train = np.concatenate((X_train,X_temp),0)
                if dataset3[n][0] == i:
                    Y_train.append(dataset1[n][1])
            else:
              count_1+= 1
              if(count_1<27):
                X_temp = data.iloc[:,:].values
                X_temp = make_data(X_temp)
                X_train = np.concatenate((X_train,X_temp),0)
                if dataset3[n][0] == i:
                    Y_train.append(dataset1[n][1])
    

X_train = np.delete(X_train,0,0)
Y_train = np.array(Y_train)

In [151]:
Y_train

array([], dtype=float64)

##Running Model

In [ ]:
def init_model():
    model = Sequential()
    initial_learning_rate = 0.001
    lr_schedule = ExponentialDecay(initial_learning_rate, decay_steps=2000, decay_rate=0.5)
    adam = Adam(learning_rate=lr_schedule)

    model.add(Conv1D(60, 10, input_shape = (40000, 74), activation = 'relu'))
    model.add(MaxPooling1D(pool_size = 3))
    model.add(Conv1D(30, 5, activation = 'relu'))
    model.add(MaxPooling1D(pool_size = 3))
    model.add(Conv1D(15, 5, activation = 'relu',dropout(0.5)))
    model.add(MaxPooling1D(pool_size = 3))
    
    model.add(Flatten())
    model.add(Dense(128, activation = 'relu'))
    model.add(Dense(1, activation = 'sigmoid'))

# Compiling the CNN
    cmodel.compile(
      optimizer=adam,
      loss="binary_crossentropy",
      metrics=['accuracy', 'AUC','Precision','Recall'])
  return model
  
  def modelFit(self, X, Y, epoch = 10):
    self.classifier.fit(X, Y, epochs=epoch)

  def modelPredict(self, X):
    return self.classifier.predict(X)

model = CNN_audio()
model.modelFit(X_upsample, Y_upsample, 1)
Y_pred = Thresholding(model.modelPredict(X_test), 0.8)
print(classification_report(Y_test,Y_pred))